In [18]:
import numpy as np
import pandas as pd
import re
import glob
import os 
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [99]:
w_stack_train=pd.read_csv("../final_scores/w_stack_train.csv")
w_stack_test=pd.read_csv("../final_scores/w_stack_test.csv")
nw_stack_train=pd.read_csv("../final_scores/nw_stack_train.csv")
nw_stack_test=pd.read_csv("../final_scores/nw_stack_test.csv")

In [100]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1
ID_test=w_stack_test["ID"]
ID_train=w_stack_train["ID"]

In [101]:
#meta features

In [102]:
meta_feat_train=pd.read_csv("../No_window/nw_meta_features/meta_train_l1l2.csv")
meta_feat_test=pd.read_csv("../No_window/nw_meta_features/meta_test_l1l2.csv")
meta_feat_train=meta_feat_train.drop("Unnamed: 0",axis=1)
meta_feat_test=meta_feat_test.drop("Unnamed: 0",axis=1)
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","Variation"],axis=1)
bmagic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)
magic_train=magic_train.drop("ID",axis=1)
magic_train=magic_train.reset_index()
magic_train=magic_train.rename(columns={"index":"ID"})
w_stack_train=w_stack_train.drop("ID",axis=1)
w_stack_train=w_stack_train.reset_index()
w_stack_train=w_stack_train.rename(columns={"index":"ID"})
w_stack_test=w_stack_test.drop("ID",axis=1)
w_stack_test=w_stack_test.reset_index()
w_stack_test=w_stack_test.rename(columns={"index":"ID"})
nw_stack_train=nw_stack_train.drop("ID",axis=1)
nw_stack_train=nw_stack_train.reset_index()
nw_stack_train=nw_stack_train.rename(columns={"index":"ID"})
nw_stack_test=nw_stack_test.drop("ID",axis=1)
nw_stack_test=nw_stack_test.reset_index()
nw_stack_test=nw_stack_test.rename(columns={"index":"ID"})

In [103]:
#merge all 

In [104]:
stack_train=w_stack_train.merge(nw_stack_train,on="ID").merge(meta_feat_train,
        on="ID").merge(magic_train,on="ID")
stack_test=w_stack_test.merge(nw_stack_test,on="ID").merge(meta_feat_test,
        on="ID").merge(magic_test,on="ID")


In [105]:
X_train=np.array(stack_train.drop("ID",axis=1))
X_test=np.array(stack_test.drop("ID",axis=1))

In [106]:
kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [107]:
#Ridge vs Lasso searching
param_test= {
    "C" : [0.1,0.3,0.5,0.7,0.9, 1,2,3,4,5],
    "penalty": ["l1","l2"]
}

In [95]:
gsearch=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=kf)
gsearch.fit(X_train,y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_


C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.92112, std: 0.03985, params: {'C': 0.1, 'penalty': 'l1'},
  mean: -1.04282, std: 0.04104, params: {'C': 0.1, 'penalty': 'l2'},
  mean: -0.89603, std: 0.04312, params: {'C': 0.3, 'penalty': 'l1'},
  mean: -1.00632, std: 0.04275, params: {'C': 0.3, 'penalty': 'l2'},
  mean: -0.89463, std: 0.04325, params: {'C': 0.5, 'penalty': 'l1'},
  mean: -0.97687, std: 0.03926, params: {'C': 0.5, 'penalty': 'l2'},
  mean: -0.89533, std: 0.04393, params: {'C': 0.7, 'penalty': 'l1'},
  mean: -0.97822, std: 0.03671, params: {'C': 0.7, 'penalty': 'l2'},
  mean: -0.89682, std: 0.04456, params: {'C': 0.9, 'penalty': 'l1'},
  mean: -0.97797, std: 0.03920, params: {'C': 0.9, 'penalty': 'l2'},
  mean: -0.89751, std: 0.04570, params: {'C': 1, 'penalty': 'l1'},
  mean: -0.98121, std: 0.04348, params: {'C': 1, 'penalty': 'l2'},
  mean: -0.90254, std: 0.04912, params: {'C': 2, 'penalty': 'l1'},
  mean: -0.98049, std: 0.04482, params: {'C': 2, 'penalty': 'l2'},
  mean: -0.90754, std: 0.05025, params: {'

In [59]:
# best model to complete

In [112]:
clf = LogisticRegression(C=0.5,penalty="l1")                     
clf.fit(X_train,y)
y_test=clf.predict_proba(X_test)

classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
sub = pd.DataFrame(y_test, columns=classes)
sub['ID'] = ID_test
sub.to_csv("../final_scores/stacking_final_full_train.csv",index=False)

In [113]:
def stacker_final(X,X_test,y,classifier):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("../final_scores/stacking_final_5folds.csv",index=False)

In [114]:
stacker_final(X_train,X_test,y,clf)

Fold 1 (2946, 80) (743, 80)
Fold 2 (2949, 80) (740, 80)
Fold 3 (2952, 80) (737, 80)
Fold 4 (2954, 80) (735, 80)
Fold 5 (2955, 80) (734, 80)


In [109]:
# which model impacts more ? 